In [1]:
import pandas as pd
import io
import numpy as np
from sklearn import preprocessing 
from sklearn import svm
from sklearn import metrics
import math

In [2]:
df = pd.read_csv("drive/My Drive/2011-capitalbikeshare-tripdata.csv")

In [3]:
print(df.head())

   Duration           Start date  ... Bike number  Member type
0      3548  2011-01-01 00:01:29  ...      W00247       Member
1       346  2011-01-01 00:02:46  ...      W00675       Casual
2       562  2011-01-01 00:06:13  ...      W00357       Member
3       434  2011-01-01 00:09:21  ...      W00970       Member
4       233  2011-01-01 00:28:26  ...      W00346       Casual

[5 rows x 9 columns]


In [4]:
# select cols
cols = [
    'End station', 'Start date', 'Start station', 'End date', 'Duration',
    'Member type']
label = ['End station']
input = df[cols].copy()

input['Start date'] = pd.to_datetime(input['Start date'],format='%Y-%m-%d %H:%M:%S')
#input['End date'] = pd.to_datetime(input['End date'],format='%Y-%m-%d %H:%M:%S')

# 時間處理改為拆年月日時分秒欄位

input['s_year'] = input['Start date'].dt.year
input['s_month'] = input['Start date'].dt.month
input['s_day'] = input['Start date'].dt.day
input['s_hour'] = input['Start date'].dt.hour
input['s_minute'] = input['Start date'].dt.minute
input['s_second'] = input['Start date'].dt.second
'''
input['e_year'] = input['End date'].dt.year
input['e_month'] = input['End date'].dt.month
input['e_day'] = input['End date'].dt.day
input['e_hour'] = input['End date'].dt.hour
input['e_minute'] = input['End date'].dt.minute
input['e_second'] = input['End date'].dt.second
'''
#input['Time cost'] = (input['End date'] - input['Start date']).astype(int)
#print(input)
input = input.drop('Start date', 1)
input = input.drop('End date', 1)
input

,End station,Start station,Duration,Member type,s_year,s_month,s_day,s_hour,s_minute,s_second
0,5th & F St NW,5th & F St NW,3548,Member,2011,1,1,0,1,29
1,14th & V St NW,14th & Harvard St NW,346,Casual,2011,1,1,0,2,46
2,Adams Mill & Columbia Rd NW,Georgia & New Hampshire Ave NW,562,Member,2011,1,1,0,6,13
3,Florida Ave & R St NW,10th & U St NW,434,Member,2011,1,1,0,9,21
4,Calvert & Biltmore St NW,Adams Mill & Columbia Rd NW,233,Casual,2011,1,1,0,28,26
...,...,...,...,...,...,...,...,...,...,...
1226762,17th & Corcoran St NW,15th & P St NW,300,Member,2011,12,31,23,41,19
1226763,15th & P St NW,Convention Center / 7th & M St NW,387,Member,2011,12,31,23,46,43
1226764,Park Rd & Holmead Pl NW,Lamont & Mt Pleasant NW,261,Member,2011,12,31,23,47,27
1226765,New York Ave & 15th St NW,21st & I St NW,2060,Member,2011,12,31,23,55,12


In [6]:
#建立日期、會員的索引
station_index = {}
member_index = {}

#車站
i = 0
for name in input["Start station"]:
  if station_index.__contains__(name) == False:
    station_index.update({name : i})
    #station_index.update({i : name})
    i+=1
print(station_index)
print(len(station_index))
#會員
i = 0
for name in input["Member type"]:
  if member_index.__contains__(name) == False:
    member_index.update({name : i})
    #station_index.update({i : name})
    i+=1
print(member_index)
print(len(member_index))

{'5th & F St NW': 0, '14th & Harvard St NW': 1, 'Georgia & New Hampshire Ave NW': 2, '10th & U St NW': 3, 'Adams Mill & Columbia Rd NW': 4, '3rd & D St SE': 5, '14th & Rhode Island Ave NW': 6, '15th & P St NW': 7, 'Van Ness Metro / UDC': 8, 'Crystal City Metro / 18th & Bell St': 9, '14th & R St NW': 10, '21st & M St NW': 11, 'Eastern Market Metro / Pennsylvania Ave & 7th St SE': 12, '16th & Harvard St NW': 13, '7th & T St NW': 14, 'Wisconsin Ave & Newark St NW': 15, 'New Hampshire Ave & T St NW': 16, '1st & M St NE': 17, 'Connecticut Ave & Newark St NW / Cleveland Park': 18, '5th & K St NW': 19, '14th & V St NW': 20, 'M St & New Jersey Ave SE': 21, 'Florida Ave & R St NW': 22, '17th & K St NW': 23, '13th & H St NE': 24, 'Lamont & Mt Pleasant NW': 25, '18th & M St NW': 26, '18th & Eads St.': 27, '19th & L St NW': 28, 'Park Rd & Holmead Pl NW': 29, '19th St & Pennsylvania Ave NW': 30, '21st & I St NW': 31, '14th St Heights / 14th & Crittenden St NW': 32, 'Potomac & Pennsylvania Ave SE': 

In [8]:
for key, value in station_index.items():
    input["Start station"] = input["Start station"].replace(key, str(value))

for key, value in station_index.items():
    input["End station"] = input["End station"].replace(key, str(value))

for key, value in member_index.items():
    input["Member type"] = input["Member type"].replace(key, str(value))

In [10]:
input = input.drop('s_year', 1)
input

,End station,Start station,Duration,Member type,s_month,s_day,s_hour,s_minute,s_second
0,0,0,3548,0,1,1,0,1,29
1,20,1,346,1,1,1,0,2,46
2,4,2,562,0,1,1,0,6,13
3,22,3,434,0,1,1,0,9,21
4,74,4,233,1,1,1,0,28,26
...,...,...,...,...,...,...,...,...,...
1226762,39,7,300,0,12,31,23,41,19
1226763,7,49,387,0,12,31,23,46,43
1226764,29,25,261,0,12,31,23,47,27
1226765,78,31,2060,0,12,31,23,55,12


In [15]:
xStartStation = input['Start station']
xDurtion = input['Duration']
xMember = input['Member type']
xYear = input['s_month']
xDay = input['s_day']
xHour = input['s_hour']
xMinute = input['s_minute']
xSecond = input['s_second']
yEndStation = input['End station']

In [36]:
Y = yEndStation.values

In [42]:
print(Y[:150])
print(len(Y))
print(Y.data.shape)

['0' '20' '4' '22' '74' '71' '7' '7' '10' '10' '4' '3' '59' '59' '14' '14'
 '2' '39' '39' '55' '55' '13' '29' '29' '3' '3' '74' '14' '20' '20' '87'
 '4' '13' '13' '41' '16' '16' '7' '7' '32' '21' '32' '23' '20' '1' '57'
 '41' '41' '26' '53' '53' '38' '2' '28' '52' '80' '3' '3' '30' '71' '25'
 '31' '62' '62' '26' '25' '8' '25' '25' '1' '35' '7' '48' '54' '22' '41'
 '37' '74' '59' '39' '71' '72' '24' '4' '6' '32' '4' '25' '24' '24' '57'
 '65' '65' '41' '20' '43' '25' '22' '79' '34' '4' '26' '11' '57' '34' '48'
 '59' '16' '46' '52' '6' '29' '6' '4' '42' '42' '13' '11' '4' '50' '7'
 '50' '50' '50' '84' '7' '39' '6' '52' '6' '27' '27' '50' '41' '3' '50'
 '50' '64' '41' '64' '38' '38' '10' '10' '10' '53' '16' '74' '86' '47']
1226767
(1226767,)


In [39]:
X = np.column_stack((xStartStation, xDurtion, xMember, xYear, xDay, xHour, xMinute, xSecond))
X

array([['0', 3548, '0', ..., 0, 1, 29],
       ['1', 346, '1', ..., 0, 2, 46],
       ['2', 562, '0', ..., 0, 6, 13],
       ...,
       ['25', 261, '0', ..., 23, 47, 27],
       ['31', 2060, '0', ..., 23, 55, 12],
       ['26', 468, '0', ..., 23, 55, 56]], dtype=object)

In [40]:
print(X.data.shape)

(1226767, 8)


#建構決策樹

In [41]:
from sklearn import tree
from sklearn import datasets
import pydotplus
from sklearn.model_selection import train_test_split

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [85]:
X_train

array([['29', 343, '0', ..., 19, 56, 1],
       ['47', 206, '0', ..., 12, 6, 20],
       ['1', 1076, '0', ..., 20, 53, 44],
       ...,
       ['108', 1930, '0', ..., 17, 9, 40],
       ['77', 1156, '1', ..., 11, 42, 53],
       ['42', 754, '0', ..., 16, 37, 10]], dtype=object)

In [86]:
y_train

array(['2', '92', '19', ..., '22', '78', '73'], dtype=object)

In [87]:
clf = tree.DecisionTreeClassifier(criterion='entropy').fit(X_train,y_train)#用entropy測試

In [88]:
clf.score(X_train,y_train)

1.0

In [89]:
clf.predict(X_test)

array(['53', '108', '18', ..., '35', '22', '61'], dtype=object)

In [90]:
clf.score(X_test,y_test)

0.2201825389709019

In [91]:
clf = tree.DecisionTreeClassifier(criterion='gini',splitter='best').fit(X_train,y_train)#用gini測試

In [92]:
clf.score(X_train,y_train)

1.0

In [93]:
clf.predict(X_test)

array(['53', '30', '35', ..., '35', '13', '77'], dtype=object)

In [94]:
clf.score(X_test,y_test)

0.2213155956971016